# Extracting nuclei from 3D image stacks

This notebook allows user to load 3D microscopy images, segment nuclei, extract windows around nuclei, perform manual QC, and save them in a format suitable for downstream ML applications.

Output files will feature three tags that serve different purposes in forming A-P-N triplets:

sampleID: Negative images will have different sampleIDs (example: Zelda, Rpb1-nc12)
stackID: Positive images will have the same stackID (example: 20211115-zld-gfp-em1-03)
nucID: Positive images will have different nucIDs (for image augmentation purposes; allows multiple images from a single nucleus)

Example filename:

zld_2021115-zld-gfp-03_nuc1_01.pkl

sampleID: zld
stack_ID: 2021115-zld-gfp-03
nucID: nuc1

In [1]:
import flymovie as fm
from flymovie.simnuc import Sim
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
from importlib import reload
import scipy.ndimage as ndi
import skimage as ski
from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
#from skimage import regionprops
def extract_nuclei(stack, lmask, box_dims, mask_dil_struct=np.ones((2,10,10))):
    def inbounds(im_shape, halflens, centroid):
        for dim in [1,2]:
            if centroid[dim] - halflens[dim] < 0:
                return False
            if centroid[dim] + halflens[dim] > im_shape[dim]:
                return False
        return True
    
    def get_box(stack, box_halflengths, box_dims, centroid):
        box = stack[
                    :,
                    (centroid[1] - box_halflengths[1]):(centroid[1] + box_halflengths[1]),
                    (centroid[2] - box_halflengths[2]):(centroid[2] + box_halflengths[2])
                ]
        return box

    regions = ski.measure.regionprops(lmask)
    box_halflengths = (
            int(box_dims[0] / 2),
            int(box_dims[1] / 2),
            int(box_dims[2] / 2), 
        )
    nuc_count = 0
    ims = []
    for region in regions:
        centroid = ([round(x) for x in region.centroid])
        if inbounds(lmask.shape, box_halflengths, centroid):
            im_box = get_box(stack, box_halflengths, box_dims, centroid)
            lmask_box = get_box(lmask, box_halflengths, box_dims, centroid)
            mask_box = np.where(lmask_box == region.label, 1, 0)
            mask_box = ndi.morphology.binary_dilation(mask_box, structure=mask_dil_struct)
            im_box = np.where(mask_box, im_box, 0)
            ims.append(im_box)
            nuc_count += 1
    
    return ims


def save_nucs(ims, bad_nucs, box_dims, sampleID, stackID, outfolder, z_range=[0]):
    def get_center_z(box):
        z, i, j = np.where(box > 0)
        return round((np.max(z) + np.min(z)) / 2)

    saved_count = 0
    nucs_saved = {}
    for nucID in range(len(ims)):
        if nucID not in bad_nucs:
            center_z = get_center_z(ims[nucID])
            start_z = center_z - int(box_dims[0] / 2)

            for z_add in z_range:
                z = start_z + z_add
                filepath = os.path.join(outfolder, '_'.join([sampleID, stackID, str(nucID), str(z)]) + '.pkl')
                im_to_save = ims[nucID][z:(z + box_dims[0])]
                if np.array_equal(im_to_save.shape, box_dims):
                    saved_count += 1
                    nucs_saved[nucID] = 1
                    with open(filepath, 'wb') as outfile:
                        pickle.dump(im_to_save, outfile)
                else:
                    pass
                    #print('Extracted image is wrong size.')
    print('Saved ' + str(saved_count) + ' images from ' + str(len(nucs_saved.keys())) + ' nuclei.')


In [378]:
box_dims = (34,100,100)
sampleID = 'zld-cterm'
stackID = '20220313-zld-mneon-em3-03'
output_folder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_nuclei/set2'
stack_file = '/Users/michaelstadler/Desktop/2022-03-13/zld-mneon-em3-03.czi'

In [379]:
stack = fm.read_czi(stack_file, swapaxes=False)
#stack = stack[4:,:,:]
fm.viewer(stack, 8)

interactive(children=(Dropdown(description='Color', options=('cividis', 'gray_r', 'magma', 'Gators', 'viridis'…

In [380]:
# Segment nuclei.
lmask = fm.segment_nuclei_3Dstack_rpb1(stack, usemax=False, sigma=4, min_nuc_center_dist=50, display=False)
#lmask = fm.labelmask_filter_objsize(lmask, 10_000, 2.5e5)
fm.viewer(lmask, 8)

interactive(children=(Dropdown(description='Color', options=('cividis', 'gray_r', 'magma', 'Gators', 'viridis'…

In [381]:
# Extract nuclei.
ims = extract_nuclei(stack, lmask, box_dims, np.ones((1,7,7)))
fm.viewer(np.array(ims).max(axis=1), 5)
fm.viewer(np.array(ims), 3)

interactive(children=(Dropdown(description='Color', options=('cividis', 'gray_r', 'magma', 'Gators', 'viridis'…

interactive(children=(Dropdown(description='Color', options=('cividis', 'gray_r', 'magma', 'Gators', 'viridis'…

In [377]:
bad_nucs = [0,1,6]

save_nucs(ims, bad_nucs, box_dims, sampleID, stackID, output_folder, z_range=[-2,0,2])

Saved 24 images from 9 nuclei.


In [385]:
# Count nuclei.

folder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_nuclei/set2'

samples = {}

for f in os.listdir(folder):
    if f[0] == '.':
        continue
    splits = f.split('_')
    key = '_'.join(splits[1:3])
    sample = splits[0]

    if sample in samples:
        if key not in samples[sample]:
            samples[sample][key] = 1
    else:
        samples[sample] = {}
        samples[sample][key] = 1

for sample in samples:
    print(sample + ': ' + str(len(samples[sample].keys())))

h2b: 482
rpb1: 462
bcd: 146
zld: 230
zld-cterm: 128
